In [2]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('D:\\Dropbox\\stempy')
sys.path.append('D:\\Dropbox\\stemplot')
    
from stemplot import *

from stempy.io import *
from stempy.clustering import *
from stempy.plot import *
from stempy.denoise import *
from stempy.utils import *
from stempy.feature import *
from stempy.spatial import *
from stempy.manifold import *

In [3]:
dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\CoWSe2_total 9 datasets\CoWSe2_20211124'+'\\'

num = 5

file_name1 = dp + 'ADF1_{}.dm4'.format(num)

# load image
img = load_image(file_name1)

img = normalize_image(img, 0, 1)

In [4]:
img1 = remove_bg(img, 5)

imshow(img1)

In [5]:
get_window_size(img)

16

## denoising

In [6]:
n_components = 32
patch_size = 32
extraction_step = 2
imgf = denoise_svd(img1, n_components, patch_size, extraction_step) 

Extracting reference patches...
done in 2.83s.
Singular value decomposition...
done in 20.38s.
Reconstructing...
done in 7.72s.


In [8]:
imshow(imgf)

In [11]:
threshold = 0.2
pts = local_max(imgf, min_distance=1, threshold=threshold)

In [14]:
file_name2 = dp + 'ADF2_{}.dm4'.format(num)

# load image
img = load_image(file_name2)

img = normalize_image(img, 0, 1)

img2 = remove_bg(img, 5)

In [15]:
imshow(img2)

In [18]:
size = 24
kp = KeyPoints(pts, img2, size)
ps = kp.extract_patches(size) 

In [19]:
imshow(ps)

In [21]:
zps = ZPs(n_max=10, size=ps.shape[1])
zps.fit(ps)
X = zps.moments
X_rot = X.rotinv() 

In [22]:
plot_pca(X, 2)

In [26]:
fg = ForceGraph8(X = X,
n_neighbors=10,
init_mode = 'pca',
num_iterations = 100,
num_negative_samples=10,
force_params1=(0, 2, 1, 1),
force_params2=(2, 5, 5, 1),
divide=0.5)

from time import time
t0 = time()
xy = fg.fit_transform(X)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [27]:
l= interactive_clusters(xy, img, kp.pts, ps, clip=False, s=1) 

One cluster has been selected.


In [28]:
len(l.ind), l.ind

(162,
 array([  51,   90,  131,  179,  234,  434,  453,  480,  536,  569,  580,
         650,  652,  741,  799,  841,  842,  843,  923,  924,  953,  954,
         964,  970,  971, 1135, 1145, 1171, 1189, 1258, 1321, 1364, 1407,
        1488, 1537, 1572, 1614, 1621, 1660, 1666, 1696, 1697, 1736, 1753,
        1791, 1962, 1969, 2010, 2055, 2056, 2125, 2170, 2223, 2271, 2351,
        2402, 2428, 2489, 2516, 2574, 2607, 2651, 2698, 2774, 2776, 2792,
        2866, 2948, 2992, 2999, 3044, 3260, 3524, 3560, 3727, 3735, 3859,
        3867, 3912, 3953, 3955, 3993, 3996, 4072, 4116, 4125, 4212, 4245,
        4259, 4477, 4511, 4551, 4593, 4641, 4646, 4689, 4722, 4767, 4812,
        4822, 4826, 4896, 4939, 4940, 4960, 4997, 5001, 5029, 5041, 5122,
        5157, 5200, 5201, 5243, 5289, 5302, 5374, 5419, 5504, 5566, 5589,
        5590, 5600, 5634, 5635, 5648, 5805, 5816, 5828, 5893, 5908, 5938,
        5943, 6025, 6042, 6069, 6086, 6172, 6208, 6209, 6285, 6347, 6392,
        6413, 6429, 6432, 6467, 

In [16]:
kp.pts

array([[ 48,   8],
       [ 78,   8],
       [121,   8],
       ...,
       [452, 504],
       [461, 504],
       [482, 504]], dtype=int64)

In [17]:
np.save('pts_Ti_25.npy', kp.pts)

In [18]:
np.save('ind_Ti_25.npy', l.ind)